# Operare su dati qualitativi

All'interno di un dataset strutturato puoi trovare due tipologie di dati
<ul>
    <li>**Variabili quantitative continue:** numeri che indicano una quantità</li>
    <li>**Variabili qualiative ordinate (ordinali):** numeri o stringhe che rappresentano delle classi che possono essere ordinate</li>
    <li>**Variabili qualitative sconnesse (nominali):** numeri e stringhe rappresentanti classi che non hanno un ordine</li>
</ul>

<img src="https://github.com/ProfAI/dl00/blob/master/2%20-%20Datasets%20e%20data%20preprocessing/res/variables.png?raw=1" width="350px">

Le variabili qualitative possono essere rappresentate anche da stringhe, in questo caso bisogna codificarle all'interno di numeri per poterle usare come input per un algoritmo di machine learning.
Cominciamo caricando il dataset di esempio, questo contiene un elenco di maglie con le seguenti caratteristiche: prezzo, taglie e colore.

In [1]:
import pandas as pd

shirts = pd.read_csv("data/shirts.csv",index_col=0)
shirts.head()

,taglia,colore,prezzo
0,S,bianco,4.99
1,M,bianco,19.99
2,XL,bianco,12.49
3,XL,bianco,14.99
4,S,bianco,14.99


**NOTA BENE**<br>
In inglese le variabili qualitative sono conosciute come *categorical variables (ordinal e nominal)*.

## Mapping delle caratteristiche ordinali
Le variabili qualitative ordinate (ordinali) possono essere ordinate pertanto è possibile rappresentarle come una lista di numeri che rappresentano la posizione dell'elemento all'interno della lista ordinata.
Una funzione non può stimare la posizione di un elemento nella lista, quindi bisogna definirla manualmente con un dizionario.

### Numpy e Scikit-learn
Utilizzando il metodo vectorize di numpy possiamo creare una funzione ottimizzata per operare sugli array, questa funzione ci permetterà di sostituire i labels con i numeri corrispondenti definiti nel dizionario.

In [18]:
import numpy as np

shirts = pd.read_csv("data/shirts.csv",index_col=0)
X = shirts.values

size_mapping = {"S":0,"M":1,"L":2,"XL":3} #dizionario che ordina le misure
fmap = np.vectorize(lambda t:size_mapping[t])
X[:,0] = fmap(X[:,0])
X[:5]

array([[0, 'bianco', 4.99],
       [1, 'bianco', 19.99],
       [3, 'bianco', 12.49],
       [3, 'bianco', 14.99],
       [0, 'bianco', 14.99]], dtype=object)

### Pandas
Per sostituire labels con i numeri nel DataFrame possiamo usare il metodo map.

In [19]:
size_mapping = {"S":0,"M":1,"L":2,"XL":3} #dizionario che ordina le misure
shirts["taglia"] = shirts["taglia"].map(size_mapping) #mappiamo la misura con il numero corrispondente
shirts.head()

,taglia,colore,prezzo
0,0,bianco,4.99
1,1,bianco,19.99
2,3,bianco,12.49
3,3,bianco,14.99
4,0,bianco,14.99


# One-hot encoding
Le variabili qualitative sconnesse (nominali) non possono essere mappate come le ordinali, poiché non presentano un ordine interno e quindi è sbagliato assegnare un numero univoco ad ognuna.
Il metodo utilizzato in questo caso è il One-hot encoding, e consiste nel creare una nuova colonna per ogni classe che assumerà il valore 1 se l'esempio appartiene alla classe, 0 nell'altro caso, queste variabili sono chiamate **variabili di comodo (dummy variables)**
<img src="https://github.com/ProfAI/dl00/blob/master/2%20-%20Datasets%20e%20data%20preprocessing/res/onehot.png?raw=1" width="500px">

### Numpy e Scikit-learn
Se il nostro dataset è un array numpy possiamo utilizzare la classe OneHotEncoder di scikit-learn. Per eseguire il One Hot Encoding su solamente una o più colonne del dataset possiamo sfruttare la classe ColumnTransformer, questa richiede in input una lista di tuple, in cui ogni tupla corrisponde ad una trasformazione da eseguire su una colonna che contiene i seguenti elementi:
1. Una stringa che indica un nome arbitrario da dare alla trasformazione
2. Il trasformatore istanziato, cioè l'oggetto che vogliamo usare per eseguire la trasformazione.
3. Una lista con gli indici delle colonne alla quale vogliamo applicare la trasformazione.

Il comportamente sulle colonne non trasformate viene definito tramite il parametro *remainder*, se questo è uguale a "drop" esse verranno rimosse, se invece è uguale a "passthrough" verranno passate immutate all'array numpy di output del ColumnTransformer.

In [20]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

X = shirts.values # Otteniamo l'array numpy corrispondente al DataFrame

ct = ColumnTransformer([("colore", OneHotEncoder(), [1])], remainder = 'passthrough')
X = ct.fit_transform(X)

X[:5] #le prime tre colonne rappresentano la classe-colore (bianco, rosso e verde), la quarta la taglia e la quinta il prezzo

array([[1.0, 0.0, 0.0, 0, 4.99],
       [1.0, 0.0, 0.0, 1, 19.99],
       [1.0, 0.0, 0.0, 3, 12.49],
       [1.0, 0.0, 0.0, 3, 14.99],
       [1.0, 0.0, 0.0, 0, 14.99]], dtype=object)

### Pandas
Se invece vogliamo eseguire il one-hot encoding su un Dataframe possiamo semplicemente usare il metodo pandas get_dummies

In [0]:
shirts = pd.get_dummies(shirts,columns=["colore"])
shirts.head()

,taglia,prezzo,size,colore_bianco,colore_rosso,colore_verde
0,S,4.99,0,1,0,0
1,M,19.99,1,1,0,0
2,XL,12.49,3,1,0,0
3,XL,14.99,3,1,0,0
4,S,14.99,0,1,0,0
